In [1]:
!sudo pip install -q transformers --upgrade

In [2]:
!sudo pip install -q auto-gptq

In [3]:
!sudo pip install lade

  Created wheel for lade: filename=lade-0.0.2-cp38-none-any.whl size=13395 sha256=fd6729fc60cb090e68afea705df4d8c007adaa4f6af8fe5ba8227b872149e943
  Stored in directory: /root/.cache/pip/wheels/15/d0/e3/32941bd183da80956b820ddc0e85c46e79955ca2e9eddaf5ba
Successfully built lade


In [4]:
import os

In [5]:
os.environ["USE_LADE"]="1"

In [6]:
import lade
lade.augment_all()
lade.config_lade(LEVEL=5, WINDOW_SIZE=7, GUESS_SET_SIZE=7, DEBUG=0) 

In [3]:
!sudo pip install -q git+https://github.com/huggingface/optimum.git

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig

In [8]:
from datasets import load_dataset
# dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

In [14]:
# Load LLaMA tokenizer
from transformers import AutoTokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,
                                          # add_eos_token=True,
                                          use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [6]:
dataset = ["auto-gptq is an easy-to-use model quantization library with user-friendly apis, based on GPTQ algorithm."]
quantization = GPTQConfig(bits=4, dataset = ["c4"], desc_act=False)

In [13]:
import torch
torch.cuda.is_available()

False

In [9]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [9]:
from peft import AutoPeftModelForCausalLM

In [16]:
merged_model = "/data/mistral/query-to-mql/exp-10/nov-20/merged-model"
model = AutoModelForCausalLM.from_pretrained(merged_model, torch_dtype=torch.float16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
new_model_name = "/data/mistral/query-to-mql/exp-10/nov-20/checkpoint-8000"
merged_model = "/data/mistral/query-to-mql/exp-10/nov-20/merged-model"

import torch
torch.cuda.empty_cache()

quant_model = AutoModelForCausalLM.from_pretrained(merged_model, device_map="auto", quantization_config=quantization)

In [17]:
context = """{
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},
                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},
                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},
                  {"ENTITY": "Country", "other names": ["country", "countries"]}],
    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},
               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile phones"], "parent": "Sub-Category"},
               {"ENTITY": "Binder", "other names": ["binders", "binder"], "parent": "Sub-Category"},
               {"ENTITY": "Corporate", "other names": ["corporates", "corporate"], "parent": "Segment"},
               {"ENTITY": "India", "other names": ["india"], "parent": "Country"},
               {"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [{"ENTITY": "Ratio",
             "other names": ["ratio", "share", "contribution", "percentage", "proportion", "contributing"]},
            {"ENTITY": "Why", "other names": ["why", "cause of", "reason for", "diagnose"]},
            {"ENTITY": "contribution_to_growth", "other names": ["contribution to growth", "growth", "grown"]},
            {"ENTITY": "kda_transactional", "other names": ["kda", "key drivers", "key driver", "drivers", "driver"]},
            {"ENTITY": "Growth Rate", "other names": ["growth rate", "growth", "grown"]},
            {"ENTITY": "correlation",
             "other names": ["associate", "associated", "association", "associations", "correlate", "correlated",
                             "correlation", "correlations", "relate", "related", "relation", "relations",
                             "relationship",
                             "relationships"]}
            ],
    "DATE VARIABLE": [{"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
    }"""

date_input = {
    "start_date": "01/01/2020",
    "end_date": "15/09/2023"
}

In [18]:
query_template_v1 = """Given the context : {context} and date reference: {date_input}, the query: {user_query}, is converted into below shown structured output.
[MQL]
"""

In [ ]:
model_inputs = tokenizer(input_text, return_tensors='pt').to(torch_device)
greedy_output = model.generate(**model_inputs, max_new_tokens=1024) #speedup obtained

In [19]:
def predict_template_query_v1(user_query):
    inp = query_template_v1.format(context=context,
                                   user_query=user_query,
                                  date_input=date_input)
    _inputs = tokenizer.encode(inp, return_tensors="pt")
    outputs = model.generate(input_ids=_inputs, max_length= 1700, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    output_new = output.split('[MQL]\n')[1]
    return output_new.split('\n[/MQL]')[0], output

In [21]:
import time

In [23]:
del model

In [ ]:
from vllm import LLM, SamplingParams

In [ ]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

In [ ]:
llm = LLM(model="facebook/opt-125m")

In [ ]:
outputs = llm.generate(prompts, sampling_params)

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

In [22]:
%%time
user_query = 'sales in nov 2020 to may 2021 vs Q3 2021'
print('user query: ', user_query)
print('-'*100)
start = time.time()
output, raw = predict_template_query_v1(user_query=user_query)
print('total time taken: ', time.time()-start)

user query:  sales in nov 2020 to may 2021 vs Q3 2021
----------------------------------------------------------------------------------------------------


AttributeError: 'MistralForCausalLM' object has no attribute 'jforward_multilevel'

In [ ]:
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [17]:
%%time
user_query = 'sales in nov 2020 to may 2021 vs Q3 2021'
print('user query: ', user_query)
print('-'*100)
start = time.time()
output, raw = predict_template_query_v1(user_query=user_query)
print('total time taken: ', time.time()-start)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  sales in nov 2020 to may 2021 vs Q3 2021
----------------------------------------------------------------------------------------------------
total time taken:  68.28918886184692
{'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}, 'DATE VARIABLE': {'nov 2020 to may 2021': [{'ENTITY': 'Order Date', 'DATE RANGE': '01/11/2020 - 31/05/2021', 'CONVERTED TIME ELEMENT': 'November 2020 to May 2021'}], 'Q3 2021': [{'ENTITY': 'Order Date', 'DATE RANGE': '01/07/2021 - 30/09/2021', 'CONVERTED TIME ELEMENT': 'Q3 2021'}]}}
----------------------------------------------------------------------------------------------------


IndexError: list index out of range

In [4]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
import torch


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


# Load base model

model_name = '/data/mistral/query-to-mql/exp-10/nov-20/merged-model'
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
#     torch_dtype=torch.bfloat16,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from datasets import load_dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")


In [ ]:
quantizer.quantize(
    input_data=dataset["train"],
    valid_data=dataset["validation"],
)